In [1]:
import os
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import math
import json
from tqdm.notebook import tqdm
import glob

# Default JHMDB

In [2]:
BONE_CONNECTIONS = [

    # Face - Neck - Shoulder
    [2,0,3],
    [2,0,4],

    # Elbow - Shoulder - Shoulder
    [7,3,4],
    [8,4,3],

    #elbow - shoulder - hip
    [7,3,5],
    [8,4,6],

    #wrist - elbow - shoulder
    [11,7,3],
    [12,8,4],

    #shoulder - hip - knee
    [3,5,9],
    [4,6,10],

    #hip - knee - ankle
    [5,9,13],
    [6,10,14],
]

In [4]:
# # ALL POSSIBLE BONE CONNECTIONS
# ALL_POSSIBLE_BONE_CONNECTIONS = []
# for b in BONE_CONNECTIONS: ALL_POSSIBLE_BONE_CONNECTIONS.append(b)
# for i in range(0,15):
#     for j in range(i+1,15):
#         for k in range(0,15):

#             if k == j or k == i: continue

#             found_match = False
#             for b in ALL_POSSIBLE_BONE_CONNECTIONS:
#                 if i in b and j in b and k in b:
#                     found_match = True
#                     break
            
#             if not found_match:
#                 ALL_POSSIBLE_BONE_CONNECTIONS.append([i,k,j])

# ALL_POSSIBLE_BONE_CONNECTIONS

[[2, 0, 3],
 [2, 0, 4],
 [7, 3, 4],
 [8, 4, 3],
 [7, 3, 5],
 [8, 4, 6],
 [11, 7, 3],
 [12, 8, 4],
 [3, 5, 9],
 [4, 6, 10],
 [5, 9, 13],
 [6, 10, 14],
 [0, 2, 1],
 [0, 3, 1],
 [0, 4, 1],
 [0, 5, 1],
 [0, 6, 1],
 [0, 7, 1],
 [0, 8, 1],
 [0, 9, 1],
 [0, 10, 1],
 [0, 11, 1],
 [0, 12, 1],
 [0, 13, 1],
 [0, 14, 1],
 [0, 5, 2],
 [0, 6, 2],
 [0, 7, 2],
 [0, 8, 2],
 [0, 9, 2],
 [0, 10, 2],
 [0, 11, 2],
 [0, 12, 2],
 [0, 13, 2],
 [0, 14, 2],
 [0, 4, 3],
 [0, 5, 3],
 [0, 6, 3],
 [0, 7, 3],
 [0, 8, 3],
 [0, 9, 3],
 [0, 10, 3],
 [0, 11, 3],
 [0, 12, 3],
 [0, 13, 3],
 [0, 14, 3],
 [0, 5, 4],
 [0, 6, 4],
 [0, 7, 4],
 [0, 8, 4],
 [0, 9, 4],
 [0, 10, 4],
 [0, 11, 4],
 [0, 12, 4],
 [0, 13, 4],
 [0, 14, 4],
 [0, 6, 5],
 [0, 7, 5],
 [0, 8, 5],
 [0, 9, 5],
 [0, 10, 5],
 [0, 11, 5],
 [0, 12, 5],
 [0, 13, 5],
 [0, 14, 5],
 [0, 7, 6],
 [0, 8, 6],
 [0, 9, 6],
 [0, 10, 6],
 [0, 11, 6],
 [0, 12, 6],
 [0, 13, 6],
 [0, 14, 6],
 [0, 8, 7],
 [0, 9, 7],
 [0, 10, 7],
 [0, 11, 7],
 [0, 12, 7],
 [0, 13, 7],
 [0, 14, 7],

In [4]:
DRIVE_DIR = "D:/JHMDB"

JOINT_POSITION_FOLDER = f"{DRIVE_DIR}/joint_positions/joint_positions"
CLASSES = os.listdir(JOINT_POSITION_FOLDER)
#remove some random folders
for c in CLASSES: 
    if c[0] == ".": CLASSES.remove(c)

print(CLASSES)

['clap', 'swing_baseball', 'pour', 'run', 'wave', 'shoot_gun', 'shoot_ball', 'throw', 'golf', 'brush_hair', 'shoot_bow', 'stand', 'sit', 'climb_stairs', 'jump', 'pullup', 'pick', 'kick_ball', 'walk', 'push', 'catch']


In [38]:
for c in CLASSES:

    print(c)

    if not os.path.exists(f"{DRIVE_DIR}/processed_joints_new/{c}"):
        os.mkdir(f"{DRIVE_DIR}/processed_joints_new/{c}")

    for i in tqdm(os.listdir(f"{JOINT_POSITION_FOLDER}/{c}")):
        
        if i[0] == ".": continue

        joint_positions = scipy.io.loadmat(f"{JOINT_POSITION_FOLDER}/{c}/{i}/joint_positions.mat")['pos_img']
        
        # uncomment this to show some bone plots

        # frame0 = f"{DRIVE_DIR}/Rename_Images/Rename_Images/{c}/{i}/00001.png"
        # plt.imshow(plt.imread(frame0))
        # plt.show()
        # for c in BONE_CONNECTIONS:
        #     # plt.figure(figsize=(15,15))
        #     plt.imshow(plt.imread(frame0))
        #     print(c)
        #     for x in c:
        #         plt.scatter(joint_positions[:,:,0][0][x], joint_positions[:,:,0][1][x])
        #     plt.show()

        bone_angles = []
        for j in range(joint_positions.shape[2]):

            frame_joint_positions = joint_positions[:,:,j]

            # frame_joint_positions[1] = frame_joint_positions[1] * -1

            one_frame_bone_angles = []
            for bone_connection in BONE_CONNECTIONS:
                
                j1 = frame_joint_positions[:,bone_connection[0]]
                j2 = frame_joint_positions[:,bone_connection[1]]
                j3 = frame_joint_positions[:,bone_connection[2]]

                bone_vector_1 = [j1[0] - j2[0], (j1[1] - j2[1])*-1]
                bone_vector_2 = [j3[0] - j2[0], (j3[1] - j2[1])*-1]

                if np.linalg.norm(bone_vector_1) == 0.0 or np.linalg.norm(bone_vector_2) == 0.0:
                    one_frame_bone_angles.append(None)
                    print("bone vector was on origin")
                    continue

                bone_unit_vector_1 = bone_vector_1 / np.linalg.norm(bone_vector_1)
                bone_unit_vector_2 = bone_vector_2 / np.linalg.norm(bone_vector_2)

                bone_angle = math.degrees(math.atan2(bone_unit_vector_1[1], bone_unit_vector_1[0]) - math.atan2(bone_unit_vector_2[1], bone_unit_vector_2[0]))

                one_frame_bone_angles.append(bone_angle)

            bone_angles.append(one_frame_bone_angles)

        bone_angles = np.asarray(bone_angles)

        bone_angle_changes = []

        for skip in range(1,15):

            one_channel_bone_changes = []

            for bone_angle_index in range(bone_angles.shape[1]):

                one_bone_change = []

                for f in range(0, bone_angles.shape[0]-skip, skip):

                    one_bone_change.append(bone_angles[f, bone_angle_index] - bone_angles[f+skip, bone_angle_index])
                
                for _ in range((bone_angles.shape[0] - len(one_bone_change)) - 1):
                    one_bone_change.append(0)

                one_channel_bone_changes.append(one_bone_change)

            bone_angle_changes.append(one_channel_bone_changes)

        # bone_angle_changes = np.asarray(bone_angle_changes)
        
        # print(np.asarray(bone_angle_changes).shape)

        # print(bone_angle_changes)
        # print(bone_angles.shape)
        # channel_first_data = []
        # for j in range(bone_angle_changes.shape[2]):
        #     channel_first_data.append(bone_angle_changes[:,:,j])
        # channel_first_data = np.asarray(channel_first_data)
        # print(bone_angle_changes[:,0,1])
        # print(bone_angles[:,0])

        # print(channel_first_data.shape)
        # print(channel_first_data[0].shape)
        # print(channel_first_data[0][0])
        # plt.imshow(bone_angle_changes[0,0])

        with open(f"{DRIVE_DIR}/processed_joints_new/{c}/{i}.json", 'w') as f:
            json.dump(bone_angle_changes, f)

clap


  0%|          | 0/46 [00:00<?, ?it/s]

swing_baseball


  0%|          | 0/55 [00:00<?, ?it/s]

pour


...

  0%|          | 0/42 [00:00<?, ?it/s]

push


  0%|          | 0/43 [00:00<?, ?it/s]

catch


  0%|          | 0/49 [00:00<?, ?it/s]

# YOLOv8 Pose

In [21]:
two_person = 0
one_person = 0
no_person = 0
for i in tqdm(os.listdir(R"D:\JHMDB\JHMDB_video\predict4\labels")):
    with open(Rf"D:\JHMDB\JHMDB_video\predict4\labels\{i}") as f:
        annotations = f.read().splitlines()

        if len(annotations) == 0: no_person += 1
        if len(annotations) == 1: one_person += 1
        if len(annotations) > 1: two_person += 1

100%|██████████| 31223/31223 [00:02<00:00, 11831.18it/s]


In [55]:
for i in tqdm(glob.glob(R"D:\JHMDB\JHMDB_video\predict4\*.mp4")):

    print(i)

    annotations = []

    f_name = i.split("\\")[-1][:-4]
    annotation_files = glob.glob(f"D:\JHMDB\JHMDB_video\predict4\labels\{f_name}*")
    for a in sorted(annotation_files, key=lambda x: int(x[:-4].split("_")[-1])):

        with open(a) as f:
            annotations.append(f.read().splitlines())

    people = {}

    for p in range(len(annotations[0])):
        people[p] = annotations[0][p]

    for a in annotations[1:]:

        for p in a:
            print(p.split(" "))
            print(len(p.split(" ")))
        break
    break

  0%|          | 0/928 [00:00<?, ?it/s]

D:\JHMDB\JHMDB_video\predict4\Alexander_pushing_the_table_push_f_cm_np1_le_bad_0.mp4
['0', '0.578125', '0.595833', '0.44375', '0.666667', '0.495486', '0.381627', '0.504413', '0.37448', '0.506271', '0.365154', '0.545595', '0.397003', '0.556207', '0.367445', '0.542434', '0.48014', '0.590412', '0.433143', '0.422086', '0.503976', '0.539464', '0.426212', '0.386648', '0.481701', '0.485722', '0.401215', '0.665391', '0.659679', '0.675866', '0.647866', '0.700935', '0.748222', '0.636077', '0.708297', '0.751847', '0.839688', '0.629298', '0.806068']
39
['0', '0.907812', '0.5', '0.184375', '1', '0.86949', '0.128401', '0.869984', '0.0935959', '0.8685', '0.0954963', '0.9343', '0.0616893', '0.932109', '0.0656297', '1', '0.191781', '0.988459', '0.173432', '0.997678', '0.425804', '0.991813', '0.385438', '0.940375', '0.630933', '0.93031', '0.548945', '1', '0.598693', '1', '0.584471', '0.982916', '0.82101', '0.998041', '0.807659', '0.976133', '0.83628', '0.966876', '0.871529']
39
